In [1]:
from fake_useragent import UserAgent
import requests

ua = UserAgent()

In [93]:
from newspaper import Article
from queue import Queue
from urllib.parse import quote
from unidecode import unidecode

def get_date(load):
    try:
        date = re.findall(
            '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?', load
        )
        return '%s-%s-%s' % (date[2], date[0], date[1])
    except Exce:
        return False


def run_parallel_in_threads(target, args_list):
    globalparas = []
    result = Queue()

    def task_wrapper(*args):
        result.put(target(*args))

    threads = [
        threading.Thread(target = task_wrapper, args = args)
        for args in args_list
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())
    globalparas = list(filter(None, globalparas))
    return globalparas


def get_article(link, news, date):
    article = Article(link)
    article.download()
    article.parse()
    article.nlp()
    lang = 'ENGLISH'
    if len(article.title) < 5 or len(article.text) < 5:
        lang = 'INDONESIA'
        print('found BM/ID article')
        article = Article(link, language = 'id')
        article.download()
        article.parse()
        article.nlp()
    return {
        'title': article.title,
        'url': link,
        'authors': article.authors,
        'top-image': article.top_image,
        'text': article.text,
        'keyword': article.keywords,
        'summary': article.summary,
        'news': news,
        'date': date,
        'language': lang,
    }

In [2]:
NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT = 0

GOOGLE_NEWS_URL = 'https://www.google.com.my/search?q={}&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{}%2Ccd_max%3A{}&tbm=nws&start={}'

def forge_url(q, start, year_start, year_end):
    global NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT
    NUMBER_OF_CALLS_TO_GOOGLE_NEWS_ENDPOINT += 1
    return GOOGLE_NEWS_URL.format(
        q.replace(' ', '+'), str(year_start), str(year_end), start
    )

In [4]:
num_articles_index = 0
url = forge_url('isu syarikat', num_articles_index, '2005', '2021')
url

'https://www.google.com.my/search?q=isu+syarikat&source=lnt&tbs=cdr%3A1%2Ccd_min%3A2005%2Ccd_max%3A2021&tbm=nws&start=0'

In [39]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [40]:
response = requests.get(url, headers = headers, timeout = 60)

In [89]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser

def extract_links(content):
    soup = BeautifulSoup(content, 'html.parser')
    # return soup
    today = datetime.now().strftime('%m/%d/%Y')
    links_list = [
        v.attrs['href'] for v in soup.find_all('a', {'class': 'l lLrAF'.split()})
    ]
    dates_list = [v.text for v in soup.find_all('div', {'class': 'dhIWPd'.split()})]
    output = []
    for (link, date) in zip(links_list, dates_list):
        try:
            date = date.split('-')
            if (
                date[1].find('hour') >= 0
                or date[1].find('minute') >= 0
                or date[1].find('分鐘') >= 0
                or date[1].find('小時') >= 0
            ):
                date[1] = today
            elif date[1].find('day') >= 0 or date[1].find('日') >= 0:
                count = date[1].split(' ')[0]
            else:
                try:
                    date[1] = parser.parse(date[1]).strftime('%m-%d-%Y')
                except:
                    date[1] = 'null'
            output.append((link, date[0].strip(), date[1]))
        except Exception as e:
            print(e)
            continue
    return output

In [90]:
# response.content

In [91]:
# with open('test.html', 'w') as fopen:
#     fopen.write(response.content.decode())

In [95]:
r = extract_links(response.content)
r

[('https://www.bharian.com.my/bisnes/lain-lain/2019/07/582523/isu-malaysia-airlines-perlu-diselesai-segera',
  'Berita Harian',
  '07-08-2019'),
 ('https://www.bharian.com.my/berita/nasional/2019/03/540041/kerajaan-pertimbang-tutup-malaysia-airlines',
  'Berita Harian',
  '03-11-2019'),
 ('https://www.malaysiakini.com/letters/506320',
  'Malaysiakini (caruman)',
  '01-07-2020'),
 ('https://www.malaysiakini.com/news/491861',
  'Malaysiakini (caruman)',
  '09-13-2019'),
 ('https://www.bharian.com.my/bisnes/lain-lain/2019/10/623152/pnb-serius-tangani-isu-tadbir-urus-syarikat-pelaburan',
  'Berita Harian',
  '10-30-2019'),
 ('https://www.bharian.com.my/berita/nasional/2019/11/627385/isu-keselamatan-penerbangan-caam-akui-wujud-kelemahan',
  'Berita Harian',
  '11-11-2019'),
 ('https://www.hmetro.com.my/mutakhir/2019/10/502972/tiga-kementerian-kaji-isu-foodpanda',
  'Harian Metro',
  '10-02-2019'),
 ('https://www.bharian.com.my/berita/nasional/2019/10/613557/grab-didenda-lebih-rm86-juta',
  